In [ ]:
!git clone https://github.com/leonardozilli/FrameTrigger.git
%cd FrameTrigger/

In [1]:
!pip install transformers[torch] datasets -q

In [ ]:
from src.fn17 import load_dataset_hf
from src.train import train, test
from src.predict import predict_triggers
from src.evaluate import compute_metrics
from src.process_data import prepare_data

In [3]:
CHECKPOINT = 'bert-base-cased'

In [4]:
dataset = load_dataset_hf(flatten=True)
tokenized_dataset = prepare_data(dataset, CHECKPOINT)
tokenized_dataset

Found cached dataset frame_net_v17 (/home/leo/.cache/huggingface/datasets/liyucheng___frame_net_v17/default/0.0.0/de659a96f78d54a00870f6f51777aee2efd20c56809cfb0ced1023448403a0ab)


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/3413 [00:00<?, ? examples/s]

Map:   0%|          | 0/326 [00:00<?, ? examples/s]

Map:   0%|          | 0/1354 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3413
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 326
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1354
    })
})

## Train

In [5]:
OUT_DIR = './models'
N_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 2e-5

In [ ]:
train(pretrained_model=CHECKPOINT, dataset=tokenized_dataset,
    epochs=N_EPOCHS, batch_size=BATCH_SIZE, lr=LEARNING_RATE, model_output_path=OUT_DIR)

## Test

In [6]:
test(OUT_DIR, tokenized_dataset['test'])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/170 [00:00<?, ?it/s]

{'test_loss': 0.3330111503601074,
 'test_accuracy': 0.8798103655146177,
 'test_precision': 0.7002832163526659,
 'test_recall': 0.7726902173913044,
 'test_f1_score': 0.7347070602674246,
 'test_runtime': 164.4104,
 'test_samples_per_second': 8.235,
 'test_steps_per_second': 1.034}

## Predict

In [8]:
sentence = "The river forms a natural line between the north and south sections of the city."

predict_triggers(OUT_DIR, sentence)

'The river* forms* a natural line* between* the north* and south* sections* of the city* .'